---

**Universidad de Costa Rica** | Escuela de Ingeniería Eléctrica

*IE0405 - Modelos Probabilísticos de Señales y Sistemas*

### `PyX` - Serie de tutoriales de Python para el análisis de datos


# `Py9` - *Bases de datos SQL con Python*

> Las bases de datos relacionales permiten...

*Fabián Abarca Calderón*

---

En este tutorial los objetivos son:
    
- Aprender los conceptos básicos de bases de datos relacionales
- Usar SQLite (una base de datos en un archivo binario)
- Usar SQLAlchemy (un ORM de Python)
- Usar Pandas para manipulación de datos
- Usar una base de datos CSV de ejemplo
- Hacer operaciones CRUD de práctica

## Bases de datos relacionales

Mapeo de la relación entre dos variables en un plano cartesiano.

### Bases de datos SQL

- PostgreSQL
- SQLite
- ...

### Esquemas

Los esquemas definen la estructura dentro de una tabla de una base de datos, incluyendo el tipo de dato, el tamaño y otras características...

### SQLite


### SQLAlchemy

SQLAlchemy es un paquete de Python para manipular bases de datos tipo SQL.

- Database driver: Psycopg2
- ORM (Object Relational Mapping): SQLAlchemy

In [ ]:
from sqlalchemy import Column

class Persona():
    """Ejemplo de clase para tabla
    
    Esta es la información de una base de datos
    de registro de usuarios.
    """
    nombre = Column(Integer, adsf)


### Creación de base de datos SQLite3

Ahora... taca taca

In [ ]:
# Creación de SQLite3

### Operaciones CRUD

Create, Read, Update, Delete

In [ ]:
# Ejemplo de CRUD

## Pandas para manipulación de bases de datos

---
### Más información

* [Página web](https://www.google.com/)
* Libro o algo
* Tutorial [w3schools](https://www.w3schools.com/python/)

---
**Universidad de Costa Rica** | Facultad de Ingeniería | Escuela de Ingeniería Eléctrica

&copy; 2023

---